In [29]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [30]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os

In [31]:
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"

In [32]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = "C:\pyspark\snowpark-fips-1.8.0-with-dependencies.jar.asc.sha256"


In [33]:
conf = SparkConf() \
    .setAppName("Example") \
    .setMaster("local[*]") \
    .set("spark.driver.extraClassPath","C:/pyspark2/*") \
    .set('spark.executor.extraClassPath', 'C:/pyspark2/*')

In [34]:
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [35]:
spark

In [36]:
database = "uber"
table = "fact_table"
password = ""
user = "postgres"
schema = "analytics"
src_driver =  "org.postgresql.Driver"
target_driver = "net.snowflake.spark.snowflake.SnowflakeDriver"

In [37]:
src_url = f"jdbc:sqlserver://localhost:5432;database={database};user={user};password={password}"

In [38]:
target_driver = f"jdbc:snowflake://maleektimz.snowflakecomputing.com/?warehouse=COMPUTE_WH&db=UBER_DATA&schema=public&role=ACCOUNTADMIN&user=MALEEKTIMZ&password="

In [39]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/uber") \
    .option("dbtable", "analytics.fact_table") \
    .option("user", "postgres") \
    .option("password", "") \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [41]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:snowflake://maleektimz.snowflakecomputing.com/?warehouse=COMPUTE_WH&db=UBER_DATA&schema=public&role=ACCOUNTADMIN&user=MALEEKTIMZ&password=") \
    .option("dbtable", "PUBLIC.FACT_TABLE") \
    .option("driver", "net.snowflake.spark.snowflake.SnowflakeDriver") \
    .load()


Py4JJavaError: An error occurred while calling o126.load.
: java.lang.ClassNotFoundException: net.snowflake.spark.snowflake.SnowflakeDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:588)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:41)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRe

In [ ]:
# Read data from PostgreSQL into a DataFrame
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:snowflake://maleektimz.snowflakecomputing.com/?warehouse=COMPUTE_WH&db=UBER_DATA&schema=public&role=ACCOUNTADMIN&user=MALEEKTIMZ&password=liltimz") \
    .option("dbtable", "analytics.fact_table") \
    .option("driver", "net.snowflake.spark.snowflake.SnowflakeDriver") \
    .load()

# Write the DataFrame to Snowflake
df.write \
    .format("snowflake") \
    .options(**sf_options) \
    .option("dbtable", "fact_table") \
    .mode("append") \
    .save()

In [ ]:
df.show()

In [ ]:
#convert to pandas for better readibility
df.limit(10).toPandas()

In [ ]:
#rename column
df = df.withColumnRenamed("store_and_fwd_flag", "store")
df.show()

In [ ]:
df.count()

In [ ]:
df = df.select("VendorID", "passenger_count_id", "do_location_id")
df.show()

In [ ]:
df.sort("passenger_count_id").show()

In [ ]:
#sorting in descending order
from pyspark.sql import functions as F
df.sort(F.desc("passenger_count_id")).show()

In [ ]:
df.printSchema()